<a href="https://colab.research.google.com/github/MattiaGatto/chatbot-MATT_AI_BOT/blob/main/chatbot_python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.projectpro.io/article/python-chatbot-project-learn-to-build-a-chatbot-from-scratch/429


In [ ]:
#Mounting the Google Drive
from google.colab import drive
drive.mount('/content/drive')
data_root='/content/drive/My Drive/project'
#Please upload the files in your drive and change the path to it accordingly.

#2 Importing Relevant Libraries
import json
import string
import random
import nltk
import numpy as np
from nltk.stem import WordNetLemmatizer
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
nltk.download("punkt")
nltk.download("wordnet")
nltk.download('omw-1.4')
!pip install googletrans==3.1.0a0
# !pip install googletrans==4.0.0-rc1
# !pip uninstall googletrans
# !git clone https://github.com/alainrouillon/py-googletrans.git
# !cd ./py-googletrans
# !git checkout origin/feature/enhance-use-of-direct-api
# !python setup.py install

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
data_root + '/Dataset/intents.json'

'/content/drive/My Drive/project/Dataset/intents.json'

In [ ]:
#Loading the Dataset: intents.json 
data_file =  open(data_root + '/Dataset/intents.json').read()
data= json.loads(data_file)

In [ ]:
import googletrans
from googletrans import Translator
translator = Translator()
translator.translate('Hi',dest='it',src='en').text

'Ciao'

In [ ]:
# import pandas as pd
# data_plus = pd.read_csv(data_root + '/Dataset/txt.txt', sep='\t', encoding = 'ISO-8859-1')
# # data_plus.info()
# data_plus=data_plus[['ArticleTitle','Question','Answer']]
# questions=[]
# response=[]
# tag=[]
# contest=''
# d={"":{"patterns":[],"responses":[]}}

# for i in range(len(data_plus)):
  
#   trans_0=''
#   try:
#     trans_0=translator.translate(str(data_plus.iloc[i,0]),dest='it',src='en').text
#   except:
#     continue
#   if (trans_0 not in d):
#      d[trans_0]={"patterns":[],"responses":[]}
  
#   trans_1=''
#   try:
#     trans_1=translator.translate(str(data_plus.iloc[i,1]),dest='it',src='en').text
#     d[trans_0]["patterns"].append(trans_1)
#   except:
#     continue
  
#   trans_2=''
#   try:
#     trans_2=translator.translate(str(data_plus.iloc[i,2]),dest='it',src='en').text
#     d[trans_0]["responses"].append(trans_2)
#   except:
#     continue
#   perc=round((i/len(data_plus))*100)
#   if perc==10 or perc==20 or perc==30 or perc==40 or perc==50 or perc==60 or perc==70 or perc==80 or perc==90 or perc==100:
#     print(perc,"%")
# d

In [ ]:
# d['Zebra']['patterns'][0]

In [ ]:
# i=0
# for intents in d:
#   structure={'context': intents,
#             'patterns': d[intents]['patterns'],
#             'responses': d[intents]['responses'],
#             'tag': intents}
  
#   data['intents'].append(structure)
#   i+=1 
# len(data['intents'])         

In [ ]:
data

{'intents': [{'context': [],
   'patterns': ['Ciao', 'Buon giorno!', 'Che cosa succede?'],
   'responses': ['Hey!', 'Ciao!', 'Salve!', 'Buon giorno!'],
   'tag': 'ciao'},
  {'context': '',
   'patterns': [],
   'responses': ['Scusa, non riesco a capirti',
    'Per favore dammi più informazioni',
    'Non sono sicuro di aver capito'],
   'tag': 'nessuna risposta'},
  {'context': '',
   'patterns': ['Qual è il tuo lavoro?',
    'Che lavoro fai?',
    'Di cosa ti occupi?'],
   'responses': ['Il mio lavoro è farti sentire che tutto va bene.',
    'Lavoro per servirti nel miglior modo possibile'],
   'tag': 'lavoro'},
  {'context': [''],
   'patterns': ['Qual è la tua età?', 'Quanti anni hai?', 'Quando sei nato?'],
   'responses': ['Sono una Matt_AI e sono nato nel 2022!🔝'],
   'tag': 'età'}]}

In [ ]:
#4 Creating data_X and data_Y
words = [] #For Bow model/ vocabulary for patterns
classes = [] #For Bow model/ vocabulary for tags 
data_X = [] #For storing each pattern
data_y = [] #For storing tag corresponding to each pattern in data_X
# Iterating over all the intents
for intent in data["intents"]:
  for pattern in intent["patterns"]:
    tokens = nltk.word_tokenize(pattern) # tokenize each pattern
    words.extend(tokens) #and append tokens to words 
    data_y.append(intent["tag"]),# appending the associated tag to each pattern
    data_X.append(pattern) #appending pattern to data_X
  # adding the tag to the classes if it's not there already
  if intent["tag"] not in classes:
    classes.append(intent["tag"])
#initializing lemmatizer to get stem of words
lemmatizer=WordNetLemmatizer()
#lemmatize all the words in the vocab and convert them to lowercase
# if the words don't appear in punctuation
words = [lemmatizer.lemmatize (word. lower()) for word in words if word not in string.punctuation] #sorting the vocab and classes in alphabetical order and taking the # set to ensure no duplicates occur
words = sorted(set(words))
classes = sorted(set(classes))

In [ ]:
#5 Text to Numbers
training=[]
out_empty=[0]*len(classes)
# creating the bag of words model
for idx, doc in enumerate(data_X):
  bow = []
  text = lemmatizer.lemmatize(doc.lower())
  for word in words:
    bow.append(1) if word in text else bow.append(0)
  # mark the index of class that the current pattern is associated
  # to
  output_row = list(out_empty)
  output_row[classes.index(data_y[idx])] = 1
  # add the one hot encoded Bow and associated classes to training
  training.append([bow, output_row])
  
#shuffle the data and convert it to an array
random.shuffle(training)
training = np.array(training, dtype=object)
# split the features and target labels
train_X = np.array(list(training[:, 0]))
train_Y= np.array(list (training[:, 1]))

In [ ]:
#6 The Neural Network Model
model= Sequential() 
model.add(Dense (128, input_shape=(len(train_X[0]),), activation="relu"))
model.add(Dropout (0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout (0.5))
model.add(Dense(len(train_Y[0]), activation="softmax")) 

adam =tf.keras.optimizers.Adam(learning_rate=0.01, decay=1e-6)
model.compile(loss="categorical_crossentropy",
      optimizer=adam,
      metrics=["accuracy"])
print(model.summary())
model.fit(x=train_X, y=train_Y, epochs=150, verbose=1)

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 128)               3200      
                                                                 
 dropout_2 (Dropout)         (None, 128)               0         
                                                                 
 dense_4 (Dense)             (None, 64)                8256      
                                                                 
 dropout_3 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 4)                 260       
                                                                 
Total params: 11,716
Trainable params: 11,716
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/150
1/1 [============================

In [ ]:
#7 Preprocessing the Input
def clean_text(text):
  tokens= nltk.word_tokenize (text) 
  tokens = [lemmatizer.lemmatize (word) for word in tokens]
  return tokens
def bag_of_words (text, vocab):
  tokens = clean_text(text)
  bow = [0]*len (vocab)
  for w in tokens:
    for idx, word in enumerate (vocab):
      if word == w:
        bow[idx] = 1
  return np.array (bow)
def pred_class(text, vocab, labels):
  bow = bag_of_words (text, vocab)
  result = model.predict(np.array([bow])) [0] #Extracting probabilities
  thresh = 0.5
  y_pred = [[indx, res] for indx, res in enumerate (result) if res > thresh]
  y_pred.sort(key=lambda x: x[1], reverse=True) #Sorting by values of probability in decreasing order
  return_list = []
  for r in y_pred:
    return_list.append(labels [ r[0]]) #Contains labels (tags) for highest probability
  return return_list
def get_response(intents_list, intents_json):
  if len(intents_list) == 0:
    result = "Scusami! Non ho capito!"
  else:
    tag = intents_list[0]
    list_of_intents = intents_json["intents"]
  for i in list_of_intents:
    if i["tag"] == tag:
      result = random.choice(i["responses"])
      break
  return result

In [ ]:
#8 Interacting with the chatbot
print("Scrivi 'esci', se non vuoi chattare con il nostro ChatBot.")
while True:
  message = input("")
  if message == "esci":
    break
  intents=pred_class(message, words, classes) 
  result = get_response(intents, data)

---
# MATT_AI_BOT

https://www.upgrad.com/blog/how-to-make-chatbot-in-python/

In [ ]:
#Mounting the Google Drive
from google.colab import drive
drive.mount('/content/drive')
data_root='/content/drive/My Drive/project'
#Please upload the files in your drive and change the path to it accordingly.
!pip install chatterbot==1.0.4
!pip install chatterbot_corpus
# !pip install git+git://github.com/gunthercox/ChatterBot.git@master
# !pip install --upgrade chatterbot_corpus
# !pip install --upgrade chatterbot
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer

!pip install googletrans==3.1.0a0
import googletrans
from googletrans import Translator
from chatterbot.trainers import ChatterBotCorpusTrainer 

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
my_bot = ChatBot(name='MATT_AI', read_only=True, logic_adapters= ['chatterbot.logic.MathematicalEvaluation', 'chatterbot.logic.BestMatch'])

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# small_talk= ['hi there!',
# 'hi!', 'how do you do?',
# 'how are you?',
# 'i\'m cool.',
# 'fine, you?',
# 'always cool.',
# 'i\'m ok',
# 'glad to hear that.',
# 'i\'m fine',
# 'glad to hear that.',
# 'I feel awesome',
# 'excellent, glad to hear that.', 'not so good',
# 'sorry to hear that.',
# 'what is your name?',
# 'i am pybot. ask me a math question, please.']
# math_talk_1 = ['pythagorean theorem', 'a squared plus b squared equals c squared.']
# math_talk_2 = ['law of cosines',
# 'c**2 = a**2 + b**2 - 2 * a *b*cos(gamma)']

# list_trainer = ListTrainer(my_bot) 
# for item in (small_talk, math_talk_1, math_talk_2): 
#   list_trainer.train(item)

# print(my_bot.get_response("hi"))
# print(my_bot.get_response("i feel awesome today"))
# print(my_bot.get_response("what's your name?"))
# print(my_bot.get_response("show me the pythagorean theorem"))
# print(my_bot.get_response("do you know the law of cosines?"))

In [ ]:
def risposta(message,MATT_AI_bot):
  trans_quest=translator.translate(str(message),dest='en',src='it').text
  resp=MATT_AI_bot.get_response(trans_quest)
  trans_resp=translator.translate(str(resp),dest='it',src='en').text
  print(trans_resp)

def MATT_AI_BOT(MATT_AI_bot):
  print("Scrivi 'esci', se non vuoi chattare con il nostro ChatBot.")
  while True:
    message = input("")
    if message == "esci":
      break
    risposta(message,MATT_AI_bot)

In [ ]:
corpus_trainer = ChatterBotCorpusTrainer (my_bot) 
corpus_trainer.train('chatterbot.corpus.english')

translator = Translator()
translator.translate('Hi',dest='it',src='en').text

MATT_AI_bot=corpus_trainer.chatbot

Training ai.yml: [####################] 100%
Training botprofile.yml: [####################] 100%
Training computers.yml: [####################] 100%
Training conversations.yml: [####################] 100%
Training emotion.yml: [####################] 100%
Training food.yml: [####################] 100%
Training gossip.yml: [####################] 100%
Training greetings.yml: [####################] 100%
Training health.yml: [####################] 100%
Training history.yml: [####################] 100%
Training humor.yml: [####################] 100%
Training literature.yml: [####################] 100%
Training money.yml: [####################] 100%
Training movies.yml: [####################] 100%
Training politics.yml: [####################] 100%
Training psychology.yml: [####################] 100%
Training science.yml: [####################] 100%
Training sports.yml: [####################] 100%
Training trivia.yml: [####################] 100%


In [ ]:
MATT_AI_BOT(MATT_AI_bot)